In [4]:
import json
import re
# from nltk.tokenize import sent_tokenize, word_tokenize


In [5]:
def load_processed_tweets_from_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

data = load_processed_tweets_from_json("gg2013_preprocessed.json")

In [ ]:
def clean_retweet_text(text):
    """Removes the 'rt @username:' prefix from a retweet, leaving only the main content."""
    # Check if the text starts with 'rt @'
    if text.startswith("rt @"):
        # Find the position of the first colon after 'rt @username'
        colon_position = text.find(":")
        # If a colon exists, return the text after it; otherwise, return the full string
        if colon_position != -1:
            return text[colon_position + 1:].strip()
    return text.strip()

In [6]:

import nltk
from nltk.corpus import words
nltk.download('words')

award_name_allowlist = ["drama", "musical", "comedy", "animated", "animation", "motion", "television", "series"]

def extract_award_name(sentence):
    """Extracts the award name and winner from the sentence based on specified conditions."""
    # Find all hyphen and colon positions
    split_positions = [i for i, char in enumerate(sentence) if char in "-:"]
    award_name = None
    winner = None

    for index, pos in enumerate(split_positions):
        # Split the sentence at the current hyphen/colon position

        if index >= 1:
            left_part = sentence[split_positions[index - 1] + 1: split_positions[index]].strip()
        else:
            left_part = sentence[:pos].strip()
        right_part = sentence[pos + 1:].strip()

        # Check if there's something on the right
        if not right_part:
            return award_name, winner

        # Split left part into words
        left_words = left_part.split()

        # Check for "best" or "award" in left part
        if any(word.lower() in left_words for word in ['best', 'award']):

            # Determine the right portion based on allowlist keywords
            if index < len(split_positions) - 1:
                next_segment = sentence[pos + 1:split_positions[index + 1]].strip()
            else:
                next_segment = right_part

            # Assign award name and winner based on allowlist
            if any(word.lower() in next_segment.lower() for word in award_name_allowlist):
                award_name = f"{left_part} - {next_segment}".strip()
                # Capture the winner if more splits remain
                if index < len(split_positions) - 2:
                    winner = sentence[split_positions[index + 1] + 1: split_positions[index + 2]].strip()
            else:
                award_name = left_part
                winner = next_segment
            break

    return [award_name, winner]












['best supporting actor in a motion picture', ' christoph waltz ']
['best supporting actor in a motion picture - comedy or picture', ' christoph waltz ']
['best supporting actor in a motion picture', ' christoph waltz']


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\18723\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [7]:
def extract_winner_info(sentence):
    """Check if the sentence is in the format '... wins ...' with 'best' or 'award' in the second part."""

    pattern = r'^(.*?)(wins|receives)(.*?)(best|award)(.*?)$'
    match = re.search(pattern, sentence, re.IGNORECASE)

    if match:

        first_part = match.group(1).strip()
        second_part = match.group(3).strip() + " " + match.group(
            4).strip()  + match.group(5).strip()
        return [first_part, second_part]

    return []

def get_combinations(input_string):
   
    words = input_string.split()
    combinations = []
    
    

    for j in range(1, len(words) + 1):
        
        combinations.append(' '.join(words[0:j]))
    
    return combinations

In [17]:
for index, tweet in enumerate(data):
    
    tweet_text = tweet['text']
    
    award_name = extract_winner_info(tweet_text)
    
    if award_name:
        win_resolutions = {
            "award" : get_combinations(award_name[1]),
            "winner": [award_name[0]],
            "confidence": 0.7
            
        }
        tweet["win_resolutions"] = win_resolutions
        
        continue
    award_name = extract_award_name(tweet_text)
    
    if award_name:
        win_resolutions = {
            "award" : get_combinations(award_name[0]),
            "winner": [award_name[1]],
            "confidence": 0.5
            
        }
        tweet["win_resolutions"] = win_resolutions

        
        

In [9]:
%pip install blis

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\18723\\AppData\\Local\\Temp\\pip-uninstall-lyo0f3pl\\numpy\\linalg\\_umath_linalg.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




  Using cached numpy-2.1.2-cp311-cp311-win_amd64.whl (12.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


In [ ]:
!python -m spacy download en_core_web_sm


In [10]:
import spacy
nlp = spacy.load("en_core_web_sm")
def get_part_before_host(sentence):
    # pattern to capture only the part before the hosting-related verbs
    pattern = r"(.*?)(?:\s+(?:is|are)\s+)?(?:hosts|host|are hosting|is hosting)"

    
    match = re.search(pattern, sentence, re.IGNORECASE)

    if match:
        # Return the part before the hosting verb
        return match.group(1).strip()

    return None


def find_hosts_from_text(text):
    
    doc = nlp(text)

    hosts = []

    for sentence in doc.sents:


        # Find the root verb of the sentence
        root_verb = [token for token in sentence if token.dep_ == "ROOT"]

        if root_verb and root_verb[0].lemma_ == "host":
            # Get the part before the hosting verb using regex function
            sentence_text = str(sentence)
            before_host = get_part_before_host(sentence_text)

            if before_host:
                # Split the part by "and" or "&" to extract multiple hosts
                
                potential_hosts = re.split(r'\s*(?:and|&|/|\\)\s*', before_host)

                hosts.extend(potential_hosts)

    return hosts



In [20]:
for index, tweet in enumerate(data):
    
    tweet_text = tweet['text']
    
    
    hosts = find_hosts_from_text(tweet_text)
    
    if hosts:
        host_resolutions = {
            "hosts": hosts,
            "confidence": 0.7
            
            
        }
        tweet["host_resolutions"] = host_resolutions
        
    
        
        

In [21]:
filename = 'parsed_data.json'

# Write data to JSON file
with open(filename, 'w') as json_file:
    json.dump(data, json_file, indent=4)